In [1]:
from IPython.display import display, Math, Latex
import numpy as np
from sympy import symbols, Function, Matrix, Rational, simplify, diff, Eq, latex
from sympy import Symbol, pi, solve


The Christoffel Symbols can be calculated using the formula:
$$ 
\Gamma^\mu_{\nu\sigma} = \frac{1}{2} g^{\mu\alpha} \left( g_{\sigma\alpha,\nu} + g_{\nu\alpha,\sigma} - g_{\nu\sigma,\alpha} \right)
$$

where $g_{\mu \nu}$ is the metric tensor. For the Freedmann-Robertson-Walker the line element is given by:
$ds^2 = -a(t)^2 \, dt^2 + a(t)^2 \left( \frac{dr^2}{1 - kr^2} + r^2 d\theta^2 + r^2 \sin^2 \theta \, d\phi^2 \right)$

Hence metric tensor can be read out as:

$g_{\mu \nu} =
\begin{pmatrix}
 -a(t)^2 & 0 & 0 & 0 \\
0 & \frac{a(t)^2}{1 - kr^2} & 0 & 0 \\
0 & 0 & a(t)^2 r^2 & 0 \\
0 & 0 & 0 & a(t)^2 r^2 \sin^2 \theta
\end{pmatrix}$


Where $a(t)$ is the scale factor as a function of time.
$r,\theta, \phi$ are the comoving spherical coordinates.
$k$ is the curvature parameter, which can be $0, +1 $ or $
−1$ corresponding to flat, closed, or open universes, respectively.


Metric for a homogeneous and isotropic spacetime written in Cartesian coordinates.
Also we will use the universe is flat $k = 0$ and hence in cartesian coordinate basis:

$g_{\mu \nu} =
\begin{pmatrix}
 -a(t)^2 & 0 & 0 & 0 \\
0 & a(t)^2 & 0 & 0 \\
0 & 0 & a(t)^2 & 0 \\
0 & 0 & 0 & a(t)^2 \theta
\end{pmatrix}$

Now the Christoffel symbols (or connections) can be calculated using the following formula:

$\large  \Gamma^\mu_{\nu\sigma} = \large  \frac{1}{2}g^{\mu\alpha}(g_{\sigma\alpha,\nu} + g_{\nu\alpha,\sigma} - g_{\nu\sigma,\alpha})$

where $g^{\mu\nu}$ is the inverse of $g_{\mu\nu}$

For more detail follow https://en.wikipedia.org/wiki/Christoffel_symbols

Setting $c=1$, the Einstein (field) equations are:

$\large R_{\mu\nu} - \frac{1}{2}g_{\mu\nu}R = 8\pi G T_{\mu\nu}$

Refer https://en.wikipedia.org/wiki/Einstein_field_equations

Greek indices run over $\{0, 1, 2, 3\}$, corresponding to $x^\mu = [t, x, y, z]$: $x^0 = t$, $x^1 = x$, $x^2 = y$, $x^3 = z$. The above is then really $4\times 4 = 16$ equations.

The Ricci curvature tensor $(R_{\mu \nu})$ and scalar ($R$)

$\large R_{\mu\nu} = \large  \Gamma^\alpha_{\mu\nu,\alpha} - \Gamma^\alpha_{\nu\alpha,\mu} + \Gamma^\alpha_{\alpha\beta}\Gamma^\beta_{\mu\nu} - \Gamma^\alpha_{\mu\beta}\Gamma^\beta_{\alpha\nu}$

and

$\large{R} = \large R^\alpha_\alpha = g^{\alpha\beta}R_{\alpha\beta}$

More detail refer: https://en.wikipedia.org/wiki/Ricci_curvature#Definition_via_local_coordinates_on_a_smooth_manifold

In [7]:


class FLRWModel:
    def __init__(self):
        # Define Cartesian coordinates
        self.x = symbols('t x y z')
        self.t = self.x[0]
        
        # Define the scale factor as a function of time
        self.a = Function('a', positive=True)(self.t)
        
        # Define the metric tensor
        self.g = Matrix([
            [-1,  0,    0,    0],
            [ 0, self.a**2,  0,    0],
            [ 0,  0,  self.a**2,   0],
            [ 0,  0,    0,  self.a**2]
        ])
        
        # Compute the inverse metric
        self.g_inv = self.g.inv()
        
        # Define the stress-energy tensor components
        self.rho = Function('rho')(self.t)
        self.P = Function('P')(self.t)
        self.u = [-1, 0, 0, 0]
        self.T = Matrix(4, 4, lambda i, j: (self.rho + self.P)*self.u[i]*self.u[j] + self.P*self.g[i, j])
        
        # Initialize Christoffel symbols and Ricci tensor
        self.Gamma = np.zeros((4, 4, 4), dtype=object)
        self.Ricci_tensor = np.zeros((4, 4), dtype=object)
        self.Ricci_scalar = None

    def latex_print(self, lhs, rhs=None):
        """Displays LaTeX-formatted equations or expressions."""
        if rhs is None:
            display(Math(f"{lhs} = {latex(simplify(lhs))}"))
        else:
            display(Math(f"{lhs} = {latex(simplify(rhs))}"))

    def compute_christoffel_symbols(self):
        """Compute the Christoffel symbols for the metric."""
        for mu in range(4):
            for nu in range(4):
                for sigma in range(4):
                    self.Gamma[mu, nu, sigma] = Rational(1, 2) * sum(
                        self.g_inv[mu, alpha] * (
                            diff(self.g[sigma, alpha], self.x[nu]) +
                            diff(self.g[nu, alpha], self.x[sigma]) -
                            diff(self.g[nu, sigma], self.x[alpha])
                        ) for alpha in range(4)
                    )

    def display_christoffel_symbols(self):
        """Display the non-zero Christoffel symbols."""
        for mu in range(4):
            for nu in range(4):
                for sigma in range(4):
                    if self.Gamma[mu, nu, sigma] != 0:
                        self.latex_print(f"\\Gamma^{{{self.x[mu]}}}_{{{self.x[nu]}{self.x[sigma]}}}", self.Gamma[mu, nu, sigma])

    def compute_ricci_tensor(self):
        """Compute the Ricci tensor using the Christoffel symbols."""
        for mu in range(4):
            for nu in range(4):
                self.Ricci_tensor[mu, nu] = (
                    sum(diff(self.Gamma[alpha, mu, nu], self.x[alpha]) for alpha in range(4)) -
                    sum(diff(self.Gamma[alpha, nu, alpha], self.x[mu]) for alpha in range(4)) +
                    sum(self.Gamma[alpha, alpha, beta] * self.Gamma[beta, mu, nu] for alpha in range(4) for beta in range(4)) -
                    sum(self.Gamma[alpha, mu, beta] * self.Gamma[beta, alpha, nu] for alpha in range(4) for beta in range(4))
                )

    def display_ricci_tensor(self):
        """Display the non-zero elements of the Ricci tensor."""
        for mu in range(4):
            for nu in range(4):
                if self.Ricci_tensor[mu, nu] != 0:
                    self.latex_print(f"R_{{{self.x[mu]}{self.x[nu]}}}", self.Ricci_tensor[mu, nu])

    def compute_ricci_scalar(self):
        """Compute the Ricci scalar by contracting the Ricci tensor with the inverse metric."""
        self.Ricci_scalar = sum(self.g_inv[alpha, beta] * self.Ricci_tensor[beta, alpha] for alpha in range(4) for beta in range(4))
        self.latex_print("R", self.Ricci_scalar)

    def run(self):
        """Run all computations and display results."""
        self.latex_print("g_{\\mu\\nu}", self.g)
        self.latex_print("g^{\\mu\\nu}", self.g_inv)
        self.latex_print("T_{\\mu\\nu}", self.T)
        
        self.compute_christoffel_symbols()
        self.display_christoffel_symbols()
        
        self.compute_ricci_tensor()
        self.display_ricci_tensor()
        
        self.compute_ricci_scalar()




In [9]:
# Create an instance of the FLRWModel class and run the computations
model = FLRWModel()
model.run()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In case of spherical coordinate system

In [12]:
import sympy as sym

class FRWCalculator:
    def __init__(self):
        # Define coordinates and metric components
        self.dim = 4
        self.t, self.r, self.theta, self.phi = sym.symbols(r't r \theta \phi')
        self.coors = [self.t, self.r, self.theta, self.phi]
        self.a = sym.Function('a')(self.t)
        self.k = sym.symbols('k')

        # FRW metric for non-flat spatial curvature
        self.metric = sym.diag(1, -self.a**2 / (1 - self.k * self.r**2),
                               -self.a**2 * self.r**2,
                               -self.a**2 * self.r**2 * sym.sin(self.theta)**2)
        self.g_inv = self.metric.inv()  # Inverse metric
        self.det_g = self.metric.det()  # Determinant of the metric

        # Allocate space for tensors
        self.gamma = sym.MutableDenseNDimArray([0] * self.dim**3, (self.dim, self.dim, self.dim))
        self.ricci_tensor = sym.MutableDenseNDimArray([0] * self.dim**2, (self.dim, self.dim))
        self.ricci_scalar = 0
        self.einstein_tensor = sym.MutableDenseNDimArray([0] * self.dim**2, (self.dim, self.dim))

    def compute_christoffel_symbols(self):
        r""" Compute Christoffel symbols \Gamma^i_{jk} """
        for i in range(self.dim):
            for j in range(self.dim):
                for k in range(self.dim):
                    term = 0
                    for l in range(self.dim):
                        term += (sym.diff(self.metric[j, l], self.coors[k]) +
                                 sym.diff(self.metric[k, l], self.coors[j]) -
                                 sym.diff(self.metric[j, k], self.coors[l])) * self.g_inv[l, i]
                    self.gamma[i, j, k] = sym.simplify(1 / 2 * term)

    def compute_ricci_tensor(self):
        """ Compute Ricci tensor R_{ij} """
        for i in range(self.dim):
            for j in range(self.dim):
                term = 0
                for k in range(self.dim):
                    term += (sym.diff(self.gamma[k, i, j], self.coors[k]) -
                             sym.diff(self.gamma[j, i, k], self.coors[k]))
                    for l in range(self.dim):
                        term += (self.gamma[k, l, i] * self.gamma[l, j, k] -
                                 self.gamma[j, l, k] * self.gamma[l, i, k])
                self.ricci_tensor[i, j] = sym.simplify(term)

    def compute_ricci_scalar(self):
        """ Compute Ricci scalar R """
        self.ricci_scalar = sym.simplify(sum(self.g_inv[i, j] * self.ricci_tensor[i, j]
                                              for i in range(self.dim) for j in range(self.dim)))

    def compute_einstein_tensor(self):
        """ Compute Einstein tensor G_{ij} """
        for i in range(self.dim):
            for j in range(self.dim):
                self.einstein_tensor[i, j] = sym.simplify(self.ricci_tensor[i, j] -
                                                          1 / 2 * self.ricci_scalar * self.metric[i, j])

    def display_results(self):
        """ Display Christoffel symbols, Ricci tensor, Ricci scalar, and Einstein tensor """
        print("\n--- Christoffel Symbols ---")
        for i in range(self.dim):
            for j in range(self.dim):
                for k in range(self.dim):
                    if self.gamma[i, j, k] != 0:
                        print(f"\u0393^{self.coors[i]}_{{{self.coors[j]}{self.coors[k]}}} = {self.gamma[i, j, k]}")

        print("\n--- Ricci Tensor ---")
        for i in range(self.dim):
            for j in range(self.dim):
                if self.ricci_tensor[i, j] != 0:
                    print(f"R_{{{self.coors[i]}{self.coors[j]}}} = {self.ricci_tensor[i, j]}")

        print("\n--- Ricci Scalar ---")
        print(f"R = {self.ricci_scalar}")

        print("\n--- Einstein Tensor ---")
        for i in range(self.dim):
            for j in range(self.dim):
                if self.einstein_tensor[i, j] != 0:
                    print(f"G_{{{self.coors[i]}{self.coors[j]}}} = {self.einstein_tensor[i, j]}")


if __name__ == "__main__":
    calculator = FRWCalculator()
    calculator.compute_christoffel_symbols()
    calculator.compute_ricci_tensor()
    calculator.compute_ricci_scalar()
    calculator.compute_einstein_tensor()
    calculator.display_results()



--- Christoffel Symbols ---
Γ^t_{rr} = -1.0*a(t)*Derivative(a(t), t)/(k*r**2 - 1)
Γ^t_{\theta\theta} = 1.0*r**2*a(t)*Derivative(a(t), t)
Γ^t_{\phi\phi} = 1.0*r**2*a(t)*sin(\theta)**2*Derivative(a(t), t)
Γ^r_{tr} = 1.0*Derivative(a(t), t)/a(t)
Γ^r_{rt} = 1.0*Derivative(a(t), t)/a(t)
Γ^r_{rr} = -1.0*k*r/(k*r**2 - 1)
Γ^r_{\theta\theta} = 1.0*r*(k*r**2 - 1)
Γ^r_{\phi\phi} = 1.0*r*(k*r**2 - 1)*sin(\theta)**2
Γ^\theta_{t\theta} = 1.0*Derivative(a(t), t)/a(t)
Γ^\theta_{r\theta} = 1.0/r
Γ^\theta_{\thetat} = 1.0*Derivative(a(t), t)/a(t)
Γ^\theta_{\thetar} = 1.0/r
Γ^\theta_{\phi\phi} = -0.5*sin(2*\theta)
Γ^\phi_{t\phi} = 1.0*Derivative(a(t), t)/a(t)
Γ^\phi_{r\phi} = 1.0/r
Γ^\phi_{\theta\phi} = 1.0/tan(\theta)
Γ^\phi_{\phit} = 1.0*Derivative(a(t), t)/a(t)
Γ^\phi_{\phir} = 1.0/r
Γ^\phi_{\phi\theta} = 1.0/tan(\theta)

--- Ricci Tensor ---
R_{tt} = (3.0*k*r**2 + 1.0*r**2*(k*r**2 - 1)*(cos(\theta)**2 - 2)*a(t)**2 + 1.0*a(t)**2 - 3.0)*Derivative(a(t), t)**2/((k*r**2 - 1)*a(t)**2)
R_{tr} = (1.0*r**2*(